In [1]:
import torch
np_loc = 0
# if torch.cuda.is_available():
#     print("GPU is available.")
#     import cupy as np
#     np_loc = 1
# else:
#     print("GPU is not available, using numpy instead.")
#     import numpy as np
    
import numpy as np

import cv2
import os
import open3d as o3d
import json
import glob
from tqdm import tqdm
from natsort import natsorted
import utils
import copy
import time

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
"""
This script performs 3D point cloud model reconstruction directly on RGBD images.
"""
def merge_one_cam(record_path, cam_id, step_size):
    try:
        print("Start merging cam {} images".format(cam_id))
        if os.path.exists(os.path.join(record_path, "camera_{}".format(cam_id))):
            record_path = os.path.join(record_path, "camera_{}".format(cam_id))
        save_path = os.path.join(record_path, 'merge')
        if not os.path.exists(save_path):
            os.mkdir(save_path)
        pose_save_path = os.path.join(record_path,'pose')
        if not os.path.exists(pose_save_path):
            os.mkdir(pose_save_path)

        intrinsic_folder = record_path

        with open(os.path.join(intrinsic_folder, 'kdc_intrinsics.txt'), 'r') as infile:
            intrinsics_dict = json.load(infile)
            print('HI')

        K = np.array(intrinsics_dict['K'])
        dist = np.array(intrinsics_dict['dist'])
        print(len(os.listdir(record_path)))
        color_img_files = natsorted(glob.glob(os.path.join(record_path, 'rgb_*.png')))
        print(len(color_img_files))
        depth_img_files = natsorted(glob.glob(os.path.join(record_path, 'depth_*.png')))
        print(len(depth_img_files))
        num_imgs = len(list(color_img_files))
        ds_imgs = int(num_imgs / step_size)
        target = o3d.geometry.PointCloud()
        succeed_list = []
        fail_list = []

        # Get image list
        last_transform = np.eye(4)
        cam_T = np.eye(4)

    #     if cam_id == 0:
    #         bbox = [620, 0, 660, 720]
    #         # bound box for camera position tuned after 05/04
    #         bound_max = [307.70884356862109, 133.03577728238881, 756.01158625186258]
    #         bound_min = [-69.869545476958933, -1512.3899056200662, 379.23399044357382]

    #     else:
    #         bbox = [550, 0, 700, 720]
    #         # bbox for camera position tuned after 11/03
    #         # bbox = [0, 0, 650, 720]
    #         bound_max = [354.69967180042113, 1573.2051127945665, 729.25]
    #         bound_min = [8.5977648813832737, -16.218270431675947, 372.11855887099176]

    #     bbox_crop = o3d.geometry.AxisAlignedBoundingBox(max_bound=bound_max, min_bound=bound_min)

        # TODO: comment off this line
        bbox = None
        for i in range(ds_imgs):
            # Get color and depth, and undistort them
            color_name = color_img_files[i * step_size]
            depth_name = depth_img_files[i * step_size]
            color = cv2.imread(color_name)
            color = cv2.cvtColor(color, cv2.COLOR_BGR2RGB)
            depth = cv2.imread(depth_name, -1)
            source = utils.rgbd2pcd(color, depth, K, bbox=bbox)

            if np_loc == 1:
                source = source.transform(np.asnumpy(cam_T))
            else:
                source = source.transform(cam_T)


            # o3d.visualization.draw_geometries([source])
            source = utils.clean_pcd(source)

            # Check if the point cloud has color data before performing ICP
            # if not source.is_empty():
            #     print(f"Skipping iteration {i} due to missing color data in the point cloud.")
            #     continue

            if i == 0:
                target = source
                succeed_list.append([0, 0, 0])
                reference_pcd = copy.deepcopy(target)
                continue
            # o3d.visualization.draw_geometries([target])



            _, transformation, fitness, inlier_rmse = utils.color_icp(source, target)
            if fitness > 0 or i<3:
                succeed_list.append([0, i * step_size, fitness, inlier_rmse])
                last_transform = np.dot(last_transform, transformation)
                np.savetxt(os.path.join(pose_save_path, '%d_%s_pose.txt' % (i * step_size, cam_id)), last_transform)
                target = source
                reference_pcd += copy.deepcopy(source).transform(last_transform)
                # print("[Frame: %4d/%4d] fitness: %f, rmse: %f" % (i * step_size, num_imgs, fitness, inlier_rmse))
                o3d.io.write_point_cloud(os.path.join(save_path, 'merge_pcd_cam{}.ply'.format(cam_id)), reference_pcd)
            else:
                fail_list.append([0, i * step_size])
                # print("[Frame: %4d/%4d] icp failed! " % (i * step_size, num_imgs))
        # reference_pcd = reference_pcd.crop(bbox_crop)
        o3d.io.write_point_cloud(os.path.join(save_path, 'merge_pcd_cam{}.ply'.format(cam_id)), reference_pcd)
    except KeyboardInterrupt:
        o3d.io.write_point_cloud('./emergency_save.ply', reference_pcd)
        raise  # re-raise the exception to halt execution




In [ ]:
merge_one_cam('D:/Projects/AnacondaFiles/APPF_codes/APPF_DATA/rs_data_3/test_plant_20230804125146', '', 2)

In [16]:
directory_path = 'D:/Projects/AnacondaFiles/APPF_codes/APPF_DATA/rs_data_2'
folders = os.listdir(directory_path)
# Loop over each folder name
for folder in folders:
    # Construct the full path for each folder using os.path.join
    full_path = os.path.join(directory_path, folder)

    # Call the function
    merge_one_cam(full_path, '', 2)

Start merging cam  images
HI
497
246
246
Start merging cam  images
HI
499
247
247
Start merging cam  images
HI
487
241
241


In [15]:
print(os.listdir("D:\Projects\AnacondaFiles\APPF_codes\APPF_DATA\CTRL_DR_DATA\Batch_test"))

['20230___']


In [48]:
start_time = time.time()
parent_folder = "D:/Projects/AnacondaFiles/APPF_codes/APPF_DATA/CTRL_DR_DATA/Batch_test"
target_folder = [i for i in os.listdir(parent_folder)]
# target_folder = [i for i in os.listdir(parent_folder) if i == '20220802']
for i in range(len(target_folder)):
    subtarget_folder = os.path.join(parent_folder, target_folder[i])
    my_list = os.listdir(subtarget_folder)
    input_path = my_list
    # input_path = []
    # for folder in my_list:
        # TODO: replace it with date of scan to be processed
        # if folder.startswith('20220520'):
        #     input_path.append(folder)

    for j in range(len(input_path)):
        record_path = os.path.join(subtarget_folder, input_path[j])
        # record_path = input('record path')
        print(record_path)
        merge_one_cam(record_path, 0, 3)
        merge_one_cam(record_path, 1, 3)
        merge_two_cams(record_path)
        segment_plants(record_path)
        print("Time taken for "+str(j)+" Folder (DAS) is "+str((time.time() - start_time))+" seconds")

D:/Projects/AnacondaFiles/APPF_codes/APPF_DATA/CTRL_DR_DATA/Batch_test\20230___\20230620182249
Start merging cam 0 images
254
254
Start merging cam 1 images
256
256
Merge two pcds begin
Time taken for 0 Folder (DAS) is 801.327467918396 seconds


In [44]:
def clean_pcd(pcd: o3d.geometry.PointCloud, nb_neighbors=10, std_ratio=0.4):
    pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=10, std_ratio=0.4)
    pcd, _ = pcd.remove_radius_outlier(nb_points=20, radius=3)
    return pcd

In [46]:
pcd_path = "D:/Projects/AnacondaFiles/APPF_codes/APPF_DATA/CTRL_DR_DATA/Batch_test/20230___/20230616115755/merge/merge_pcd_cam0.ply"
pcd = o3d.io.read_point_cloud(pcd_path)
# pcd = clean_pcd(pcd)
o3d.visualization.draw_geometries([pcd])

In [5]:
pcd_path = "D:/Projects/AnacondaFiles/APPF_codes/CSIRO_APPF_Hyperspec/RS_py/data/test_plant_sv1/merge/merge_pcd_cam.ply"
pcd = o3d.io.read_point_cloud(pcd_path)
# pcd = clean_pcd(pcd)
o3d.visualization.draw_geometries([pcd])

In [7]:
pcd_path = "D:/Projects/AnacondaFiles/APPF_codes/CSIRO_APPF_Hyperspec/RS_py/data/pointclouds/merge_pcd_cam_rsd13_3.ply"
pcd = o3d.io.read_point_cloud(pcd_path)
# pcd = clean_pcd(pcd)
o3d.visualization.draw_geometries([pcd])

In [8]:
pcd_path = "D:/Projects/AnacondaFiles/APPF_codes/CSIRO_APPF_Hyperspec/RS_py/data/pointclouds/merge_pcd_cam_rs13_1.ply"
pcd = o3d.io.read_point_cloud(pcd_path)
# pcd = clean_pcd(pcd)
o3d.visualization.draw_geometries([pcd])

In [ ]:
pcd_path = "D:/Projects/AnacondaFiles/APPF_codes/CSIRO_APPF_Hyperspec/RS_py/data/pointclouds/merge_pcd_cam_rs13_1.ply"
pcd = o3d.io.read_point_cloud(pcd_path)
# pcd = clean_pcd(pcd)
o3d.visualization.draw_geometries([pcd])

In [3]:
# D:\Projects\AnacondaFiles\APPF_codes\APPF_DATA\rs_data_pt\test_plant_20230721150535\merge
pcd_path = "D:/Projects/AnacondaFiles/APPF_codes/APPF_DATA/rs_data_pt/test_plant_rs13_4/merge/merge_pcd_cam.ply"
pcd = o3d.io.read_point_cloud(pcd_path)
# pcd = clean_pcd(pcd)
o3d.visualization.draw_geometries([pcd])

In [4]:
pcd_path = "D:/Projects/AnacondaFiles/APPF_codes/APPF_DATA/rs_data_pt2/test_plant_20230726155944/merge/merge_pcd_cam.ply"
pcd = o3d.io.read_point_cloud(pcd_path)
# pcd = clean_pcd(pcd)
o3d.visualization.draw_geometries([pcd])

In [5]:
pcd_path = "D:/Projects/AnacondaFiles/APPF_codes/APPF_DATA/rs_data_pt3/test_plant_20230804135144/merge/merge_pcd_cam.ply"
#test_plant_20230804125146, test_plant_20230804135144, test_plant_20230804141352
pcd = o3d.io.read_point_cloud(pcd_path)
# pcd = clean_pcd(pcd)
o3d.visualization.draw_geometries([pcd])

In [6]:
pcd_path = "D:/Projects/AnacondaFiles/APPF_codes/APPF_DATA/rs_data_4_pt/test_plant_20230808133911/merge/merge_pcd_cam.ply"
#test_plant_20230808133429, test_plant_20230808133525, test_plant_20230808133813, test_plant_20230808133911
pcd = o3d.io.read_point_cloud(pcd_path)
# pcd = clean_pcd(pcd)
o3d.visualization.draw_geometries([pcd])

In [14]:
root_path = 'D:/Projects/AnacondaFiles/APPF_codes/APPF_DATA/rs_data_5_pt/' 
test_list = os.listdir(root_path)


for i in test_list:
    pcd = o3d.io.read_point_cloud(root_path+i+'/merge/merge_pcd_cam.ply')
    # pcd = clean_pcd(pcd)
    o3d.visualization.draw_geometries([pcd])

In [3]:
pcd_path = "D:/Projects/AnacondaFiles/APPF_codes/APPF_DATA/rs_data_5_pt/test_plant_20230809133659/merge/merge_pcd_cam.ply"
#test_plant_20230808133429, test_plant_20230808133525, test_plant_20230808133813, test_plant_20230808133911
pcd = o3d.io.read_point_cloud(pcd_path)
# pcd = clean_pcd(pcd)
o3d.visualization.draw_geometries([pcd])

In [ ]:
num_points = len(np.asarray(pcd.points))

In [2]:
pcd_path = "D:/Projects/AnacondaFiles/APPF_codes/APPF_DATA/rs_data_5_pt/test_plant_20230809133659/merge/cropped_plant.ply"
#test_plant_20230808133429, test_plant_20230808133525, test_plant_20230808133813, test_plant_20230808133911
pcd = o3d.io.read_point_cloud(pcd_path)
# pcd = clean_pcd(pcd)
num_points = len(np.asarray(pcd.points))
print(num_points)
o3d.visualization.draw_geometries([pcd])

45733169


In [5]:
pcd_path = "D:/Projects/AnacondaFiles/APPF_codes/APPF_DATA/rs_data_5_pt/test_plant_20230809133659/merge/downsampled_pcd.ply"
#test_plant_20230808133429, test_plant_20230808133525, test_plant_20230808133813, test_plant_20230808133911
pcd = o3d.io.read_point_cloud(pcd_path)
# pcd = clean_pcd(pcd)
num_points = len(np.asarray(pcd.points))
print(num_points)
o3d.visualization.draw_geometries([pcd])

6470522


In [8]:
pcd_clean = clean_pcd(pcd)
o3d.visualization.draw_geometries([pcd_clean])

In [10]:
num_points = len(np.asarray(pcd_clean.points))
print(num_points)

5439796


In [78]:
o3d.visualization.draw_geometries([pcd_clean])

In [79]:
o3d.io.write_point_cloud('D:/Projects/AnacondaFiles/APPF_codes/APPF_DATA/rs_data_5_pt/test_plant_20230809133659/merge/cleaned_ds_plant.ply', cropped_pcd_1)

True

## Preprocessing

In [40]:
def green_filter_pcd(pcd):
    xyz = np.asarray(pcd.points)
    rgb = np.asarray(pcd.colors)

    rgb_new = np.expand_dims(rgb, axis=0)
    rgb_new = (255 * rgb_new).astype(np.uint8)
    hsv = cv2.cvtColor(rgb_new, cv2.COLOR_RGB2HSV)
    mask = cv2.inRange(hsv, (35, 40, 40), (70, 255, 255))
    
    
#         # Define the HSV range for brown
#     lower_brown = (0, 50, 40)
#     upper_brown = (20, 255, 180)

#     # Define the HSV range for white
#     lower_white = (0, 0, 200)
#     upper_white = (180, 55, 255)

#     # Mask for brown
#     brown_mask = cv2.inRange(hsv, lower_brown, upper_brown)

#     # Mask for white
#     white_mask = cv2.inRange(hsv, lower_white, upper_white)

#     # Combine masks to remove both brown and white
#     mask = cv2.bitwise_or(brown_mask, white_mask)

#     # To remove the colors, you'd typically invert the mask
#     mask = cv2.bitwise_not(mask)




    green_xyz = xyz[mask[0].astype(bool)]
    colors = rgb[mask[0].astype(bool)]

    pcd_green = o3d.geometry.PointCloud()
    pcd_green.points = o3d.utility.Vector3dVector(green_xyz)
    pcd_green.colors = o3d.utility.Vector3dVector(colors)
    return pcd_green

In [37]:
def clean_pcd(pcd: o3d.geometry.PointCloud, nb_neighbors=10, std_ratio=0.4):
    pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=10, std_ratio=0.4)
    pcd, _ = pcd.remove_radius_outlier(nb_points=20, radius=3)
    return pcd

In [15]:
# A funtion to pick points for creating bounding boxes
def pick_points(pcd):
    print("")
    print("1) Please pick at least three correspondences using [shift + left click]")
    print("   Press [shift + right click] to undo point picking")
    print("2) After picking points, press 'Q' to close the window")
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window()
    vis.add_geometry(pcd)
    vis.run()  # user picks points
    vis.destroy_window()
    print("")
    return vis.get_picked_points()

In [9]:
def vis_rot_tree(pcd):
    A = np.array([-0.81830001, -61.03139877, 1.49220002])
    B = np.array([-0.90350002, -61.10419846, 3.46230006])
    AB = B - A
    AB /= np.linalg.norm(AB)  # normalize AB
    rotation_matrix_to_align_AB_with_z = o3d.geometry.get_rotation_matrix_from_xyz((AB[0], AB[1], AB[2]))
    angle_rad = np.deg2rad(5)  # rotating by 5 degrees in each step
    rotation_about_z = np.array([[np.cos(angle_rad), -np.sin(angle_rad), 0],
                                 [np.sin(angle_rad), np.cos(angle_rad), 0],
                                 [0, 0, 1]])
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(pcd)

    while True:
        # subtract A from all points
        pcd.points = o3d.utility.Vector3dVector(np.asarray(pcd.points) - A)
        # align AB with z-axis
        pcd.rotate(rotation_matrix_to_align_AB_with_z, center=(0, 0, 0))
        # rotate about z-axis
        pcd.rotate(rotation_about_z, center=(0, 0, 0))
        # undo alignment
        pcd.rotate(rotation_matrix_to_align_AB_with_z.T, center=(0, 0, 0))
        # add A back to all points
        pcd.points = o3d.utility.Vector3dVector(np.asarray(pcd.points) + A)

        vis.update_geometry(pcd)
        vis.poll_events()
        vis.update_renderer()

        time.sleep(0.05)  # control the speed of rotation

In [2]:
root_path = 'D:/Projects/AnacondaFiles/APPF_codes/APPF_DATA/rs_data_5_pt/' 
test_list = os.listdir(root_path)


In [3]:
test_list = ['test_plant_20230809133757']

['test_plant_20230809132457',
 'test_plant_20230809132913',
 'test_plant_20230809133659',
 'test_plant_20230809133757',
 'test_plant_20230809134126',
 'test_plant_20230809134757']

In [16]:
root_path = 'D:/Projects/AnacondaFiles/APPF_codes/APPF_DATA/rs_data_5_pt/test_plant_20230809133659/merge/merge_pcd_cam.ply'
pcd = o3d.io.read_point_cloud(root_path)
# pcd = clean_pcd(pcd)
o3d.visualization.draw_geometries([pcd])

In [17]:
picked_points = pick_points(pcd)


1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window
[Open3D INFO] Picked point #12696143 (-1.2e+02, -6.1e+02, 7.1e+02) to add in queue.
[Open3D INFO] Remove picked point #12696143 from pick queue.
[Open3D INFO] Picked point #25879276 (-2.4e+02, -5.5e+02, 7.2e+02) to add in queue.
[Open3D INFO] Picked point #24727128 (3.1e+02, -5.2e+02, 8.7e+02) to add in queue.
[Open3D INFO] Remove picked point #24727128 from pick queue.
[Open3D INFO] Remove picked point #25879276 from pick queue.
[Open3D INFO] Picked point #8908049 (-2.6e+02, -7.8e+02, 7.1e+02) to add in queue.
[Open3D INFO] Picked point #42055812 (1.7e+02, -8.2e+02, 6.0e+02) to add in queue.
[Open3D INFO] Picked point #4091265 (2.0e+02, -4.0e+02, 6.8e+02) to add in queue.
[Open3D INFO] Remove picked point #4091265 from pick queue.
[Open3D INFO] Picked point #917594 (2.1e+02, -3.7e+02, 5.9e+02) to add in qu

In [18]:
# Convert the indices to points
picked_points_coords = np.asarray(pcd.points)[picked_points]
print(picked_points_coords)

[[-263.74922088 -775.23270743  709.41897476]
 [ 171.66667349 -819.5215095   601.48191783]
 [ 209.67789931 -368.69878894  587.62692774]
 [-231.84755225 -375.68199567  721.88663652]
 [  23.67776295 -427.36312609  293.54245446]]


In [73]:
# 1 and 4 are opp points 1 is usally min bound and 4 with z of 3 is max bound
# bbox = o3d.geometry.AxisAlignedBoundingBox(max_bound= [  582.62714742,  -829.0915994,   890], min_bound=[ -500.88532766, -1600.62307332,   267.01971788])
# Last coordinate is usually the z-axis
bbox = o3d.geometry.AxisAlignedBoundingBox(max_bound= [ 210, -368 ,   570], min_bound=[ -263,  -775,   300])
# Now you can crop the point cloud using this bounding box
cropped_pcd_1 = pcd.crop(bbox)

# Visualize the cropped point cloud
o3d.visualization.draw_geometries([cropped_pcd_1])

In [64]:
cropped_pcd_1.points

std::vector<Eigen::Vector3d> with 5848548 elements.
Use numpy.asarray() to access data.

In [74]:
pcd_green = green_filter_pcd(cropped_pcd_1)

In [75]:
o3d.visualization.draw_geometries([pcd_green])

In [76]:
pcd_clean_baby = clean_pcd_baby(pcd_green)

In [82]:
o3d.visualization.draw_geometries([pcd_clean_baby])

In [80]:
o3d.io.write_point_cloud('D:/Projects/AnacondaFiles/APPF_codes/APPF_DATA/rs_data_5_pt/test_plant_20230809133659/merge/cleaned_baby_plants.ply', cropped_pcd_1)

True

In [59]:
def clean_pcd_baby(pcd: o3d.geometry.PointCloud, nb_neighbors=10, std_ratio=0.4):
    # pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=7, std_ratio=0.4)
    pcd, _ = pcd.remove_radius_outlier(nb_points=2, radius=2.5)
    return pcd

In [40]:

o3d.io.write_point_cloud('D:/Projects/AnacondaFiles/APPF_codes/APPF_DATA/rs_data_5_pt/test_plant_20230809133659/merge/cropped_plant.ply', cropped_pcd_1)

True

In [3]:
pcd_path = 'D:/Projects/AnacondaFiles/APPF_codes/APPF_DATA/rs_data_5_pt/test_plant_20230809133659/merge/cropped_plant.ply'
pcd = o3d.io.read_point_cloud(pcd_path)

In [4]:
# pcd_clean = clean_pcd(cropped_pcd_1)
o3d.visualization.draw_geometries([pcd])

In [6]:
import open3d as o3d

# Read the point cloud
pcd_path = 'D:/Projects/AnacondaFiles/APPF_codes/APPF_DATA/rs_data_5_pt/test_plant_20230809133659/merge/downsampled_pcd.ply'
pcd = o3d.io.read_point_cloud(pcd_path)

# Voxel grid downsampling
voxel_size = 0.8  # Adjust this value based on your requirements
downsampled_pcd = pcd.voxel_down_sample(voxel_size=voxel_size)

# Visualize the downsampled point cloud
o3d.visualization.draw_geometries([downsampled_pcd])

In [7]:

o3d.io.write_point_cloud('D:/Projects/AnacondaFiles/APPF_codes/APPF_DATA/rs_data_5_pt/test_plant_20230809133659/merge/downsampled_pcd.ply', downsampled_pcd)

True

In [5]:
pcd_path = 'D:/Projects/AnacondaFiles/APPF_codes/APPF_DATA/rs_data_5_pt/test_plant_20230809133659/merge/downsampled_pcd.ply'
pcd = o3d.io.read_point_cloud(pcd_path)

o3d.visualization.draw_geometries([pcd])

In [ ]:
# Setting the Alpha Value
alpha = 0.3

# Create the Alpha Shape as a mesh
tree_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
tree_mesh.compute_vertex_normals()  # Compute normals for better visualization

# Visualize the mesh
o3d.visualization.draw_geometries([tree_mesh])